## Interpolates a folder of single-frame Sacks-like calendars to a reference grid

In [29]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm
import os
import re
import copy
import datetime

In [30]:
calname = "Sacks_ZARC_fill_fill_120d"
infolder = "calendars_sacks/"+calname+"/"
outfolder = "merra_calendars/"+calname+"/"
reffname = "D:/data/MERRA/MERRA2_300.statD_2d_slv_Nx.20080101.nc4.nc4"

infsuf = ".crop.calendar.fill.nc"
crops = ["Maize","Soybeans","Cotton"]

In [31]:
# Create output folder if it doesn't exist
if not os.path.exists(outfolder): os.makedirs(outfolder, exist_ok=True)

In [32]:
# Open reference array
# refarr = xr.open_dataarray(reffname)
refds = xr.open_dataset(reffname)
refarr = refds["T2MMEAN"]
print(refarr)

<xarray.DataArray 'T2MMEAN' (time: 1, lat: 302, lon: 559)>
[168818 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -62.0 -61.5 -61.0 -60.5 -60.0 ... 87.0 87.5 88.0 88.5
  * lon      (lon) float64 -175.0 -174.4 -173.8 -173.1 ... 172.5 173.1 173.8
  * time     (time) datetime64[ns] 2008-01-01T12:00:00
Attributes:
    long_name:       2-meter_air_temperature
    units:           K
    fmissing_value:  1000000000000000.0
    standard_name:   2-meter_air_temperature
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0
    valid_range:     [-1.e+15  1.e+15]
    origname:        T2MMEAN
    fullnamepath:    /T2MMEAN


In [33]:
# Loop here
for crop in crops:
    # Open the dataset
    infname = infolder + crop + infsuf
    inarr = xr.open_dataset(infname)
    
    # Rename coordinates
    inarr = inarr.rename({"latitude":"lat", "longitude":"lon"})

    #Actual interpolation
#     outarr = inarr.interp(latitude=refarr['latitude'],longitude=refarr['longitude'],method = 'nearest')
    outarr = inarr.interp(lat=refarr['lat'],lon=refarr['lon'],method = 'nearest')

    # Output
    outfname = outfolder + crop + infsuf
    outarr.to_netcdf(outfname)